In [1]:
%load_ext autoreload
%autoreload 2
from birdclef.utils import get_spark
from pyspark.sql import functions as F

spark = get_spark()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/27 20:49:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/27 20:49:18 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [44]:
path = "gs://dsgt-clef-birdclef-2024/data/processed/birdclef-2024/asbfly.parquet"
df = spark.read.parquet(path).cache()
df.printSchema()
df.show()
df.count()

24/05/27 21:19:46 WARN CacheManager: Asked to cache already cached data.


root
 |-- name: string (nullable = true)
 |-- chunk_5s: long (nullable = true)
 |-- embedding: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- logits: array (nullable = true)
 |    |-- element: float (containsNull = true)

+------------+--------+--------------------+--------------------+
|        name|chunk_5s|           embedding|              logits|
+------------+--------+--------------------+--------------------+
|XC134896.ogg|       0|[-0.01697171, -0....|[4.367853, -15.69...|
|XC134896.ogg|       1|[0.08178271, -0.1...|[5.7584195, -14.0...|
|XC134896.ogg|       2|[0.15756801, -0.1...|[5.41736, -14.507...|
|XC134896.ogg|       3|[0.07789261, -0.1...|[7.383127, -14.17...|
|XC134896.ogg|       4|[0.0338157, -0.11...|[5.162613, -13.20...|
|XC134896.ogg|       5|[0.027517725, -0....|[4.7014565, -15.4...|
|XC164848.ogg|       0|[-0.037761074, 0....|[-1.5938205, -12....|
|XC164848.ogg|       1|[0.04028651, -0.0...|[-0.54023355, -11...|
|XC164848.ogg|       2|

924

In [6]:
species_text = "asbfly, ashdro1, ashpri1, ashwoo2, asikoe2, asiope1, aspfly1, aspswi1, barfly1, barswa, bcnher, bkcbul1, bkrfla1, bkskit1, bkwsti, bladro1, blaeag1, blakit1, blhori1, blnmon1, blrwar1, bncwoo3, brakit1, brasta1, brcful1, brfowl1, brnhao1, brnshr, brodro1, brwjac1, brwowl1, btbeat1, bwfshr1, categr, chbeat1, cohcuc1, comfla1, comgre, comior1, comkin1, commoo3, commyn, compea, comros, comsan, comtai1, copbar1, crbsun2, cregos1, crfbar1, crseag1, dafbab1, darter2, eaywag1, emedov2, eucdov, eurbla2, eurcoo, forwag1, gargan, gloibi, goflea1, graher1, grbeat1, grecou1, greegr, grefla1, grehor1, grejun2, grenig1, grewar3, grnsan, grnwar1, grtdro1, gryfra, grynig2, grywag, gybpri1, gyhcaf1, heswoo1, hoopoe, houcro1, houspa, inbrob1, indpit1, indrob1, indrol2, indtit1, ingori1, inpher1, insbab1, insowl1, integr, isbduc1, jerbus2, junbab2, junmyn1, junowl1, kenplo1, kerlau2, labcro1, laudov1, lblwar1, lesyel1, lewduc1, lirplo, litegr, litgre1, litspi1, litswi1, lobsun2, maghor2, malpar1, maltro1, malwoo1, marsan, mawthr1, moipig1, nilfly2, niwpig1, nutman, orihob2, oripip1, pabflo1, paisto1, piebus1, piekin1, placuc3, plaflo1, plapri1, plhpar1, pomgrp2, purher1, pursun3, pursun4, purswa3, putbab1, redspu1, rerswa1, revbul, rewbul, rewlap1, rocpig, rorpar, rossta2, rufbab3, ruftre2, rufwoo2, rutfly6, sbeowl1, scamin3, shikra1, smamin1, sohmyn1, spepic1, spodov, spoowl1, sqtbul1, stbkin1, sttwoo1, thbwar1, tibfly3, tilwar1, vefnut1, vehpar1, wbbfly1, wemhar1, whbbul2, whbsho3, whbtre1, whbwag1, whbwat1, whbwoo2, whcbar1, whiter2, whrmun, whtkin2, woosan, wynlau1, yebbab1, yebbul3, zitcis1"
species = species_text.split(", ")
len(species), species[:3]

(182, ['asbfly', 'ashdro1', 'ashpri1'])

In [9]:
len(df.first().logits)

182

In [34]:
# compute the sigmoid of the columns and only keep the rows that are above a certain threshold
def sigmoid_udf(x):
    return 1 / (1 + F.exp(-F.col(x)))


exploded = (
    df.select(
        F.concat_ws("_", "name", "chunk_5s").alias("id"),
        F.arrays_zip("logits", F.lit(species).alias("species")).alias("logits"),
    )
    .withColumn("exploded", F.explode("logits"))
    .select("id", "exploded.*")
    .withColumn("sigmoid", sigmoid_udf("logits"))
).cache()
exploded.show(n=10)
exploded.describe().show()

+--------------+----------+-------+--------------------+
|            id|    logits|species|             sigmoid|
+--------------+----------+-------+--------------------+
|XC134896.ogg_0|  4.367853| asbfly|  0.9874803002619361|
|XC134896.ogg_0|-15.698274|ashdro1|1.521690427156765...|
|XC134896.ogg_0| -16.88644|ashpri1|4.637801412704439E-8|
|XC134896.ogg_0|-18.275568|ashwoo2|1.156169890980329...|
|XC134896.ogg_0|-13.118709|asikoe2|2.007318878010537...|
|XC134896.ogg_0| -19.20848|asiope1|4.548443094786813...|
|XC134896.ogg_0|-15.332689|aspfly1|2.193297559658960...|
|XC134896.ogg_0|-20.678583|aspswi1|1.045695998256712...|
|XC134896.ogg_0|-14.366962|barfly1|5.761124273220311E-7|
|XC134896.ogg_0|-10.036581| barswa|4.376724691542048...|
+--------------+----------+-------+--------------------+
only showing top 10 rows



+-------+--------------+---------+-------+--------------------+
|summary|            id|   logits|species|             sigmoid|
+-------+--------------+---------+-------+--------------------+
|  count|        168168|   168168| 168168|              168168|
|   mean|          NULL|-Infinity|   NULL|0.002841634182035...|
| stddev|          NULL|      NaN|   NULL|0.045182425840795626|
|    min|XC134896.ogg_0|-Infinity| asbfly|                 0.0|
|    max|XC858110.ogg_9| 9.859435|zitcis1|  0.9999477508707315|
+-------+--------------+---------+-------+--------------------+



In [41]:
# median absolute deviation of the sigmoid column
median = exploded.approxQuantile("sigmoid", [0.5], 0.001)[0]
k = 1.4826
mad = (
    exploded.withColumn("diff", F.abs(F.col("sigmoid") - median)).approxQuantile(
        "diff", [0.5], 0.001
    )[0]
    * k
)
print(median, mad)
scored = exploded.withColumn("score", (F.col("sigmoid") - median) / mad).cache()
scored.withColumn("keep", F.col("score") > 3).groupBy("keep").count().show()
filtered = scored.where("score > 3")
filtered.show()

2.488640727539945e-06 3.6759336193415833e-06


24/05/27 21:18:41 WARN CacheManager: Asked to cache already cached data.


+-----+------+
| keep| count|
+-----+------+
| true| 52830|
|false|115338|
+-----+------+

+--------------+-----------+-------+--------------------+------------------+
|            id|     logits|species|             sigmoid|             score|
+--------------+-----------+-------+--------------------+------------------+
|XC134896.ogg_0|   4.367853| asbfly|  0.9874803002619361| 268633.2001278307|
|XC134896.ogg_0| -10.036581| barswa|4.376724691542048...| 11.22942100224165|
|XC134896.ogg_0| -10.922386|blrwar1|1.804928982406959...|  4.23311482412373|
|XC134896.ogg_0| -10.468658|bwfshr1|2.841234398092328...| 7.052277309084455|
|XC134896.ogg_0|  -9.644466| comgre|6.477886397038218E-5| 16.94541569387735|
|XC134896.ogg_0| -11.042306|commoo3|1.600960095374503...|3.6782384086214543|
|XC134896.ogg_0| -7.2154408|eaywag1|7.346064630671653E-4| 199.1651368478082|
|XC134896.ogg_0|-11.1132555| eurcoo|1.491310182895336...|3.3799470795772506|
|XC134896.ogg_0| -10.741207|graher1|2.163433856135454...| 5.20

In [45]:
scored.where("sigmoid > 0.5").groupBy("species").count().show()

+-------+-----+
|species|count|
+-------+-----+
|brwowl1|    8|
|putbab1|    3|
| rewbul|    2|
| comsan|    8|
|whtkin2|    2|
|purher1|    7|
|asikoe2|    1|
| litegr|    2|
| asbfly|  337|
| bcnher|    8|
|comior1|    3|
| lirplo|    8|
| grnsan|    1|
|grnwar1|    2|
|grewar3|    8|
| comgre|    8|
|commoo3|    1|
+-------+-----+



In [52]:
predictions = (
    scored.where("sigmoid > 0.5")
    .groupBy("id")
    .agg(F.collect_list("species").alias("predicted"))
)
# rows without predictions out of the total
scored.select("id").distinct().join(
    predictions.select("id"), on="id", how="anti"
).count(), scored.select("id").distinct().count()

(567, 924)

In [57]:
predictions.orderBy(F.size("predicted").desc()).show(truncate=False)

+---------------+-----------------------------------------------------------+
|id             |predicted                                                  |
+---------------+-----------------------------------------------------------+
|XC499134.ogg_22|[bcnher, brwowl1, comgre, comsan, grewar3, lirplo, purher1]|
|XC596043.ogg_17|[bcnher, brwowl1, comgre, comsan, grewar3, lirplo, purher1]|
|XC716737.ogg_6 |[bcnher, brwowl1, comgre, comsan, grewar3, lirplo, purher1]|
|XC749597.ogg_2 |[bcnher, brwowl1, comgre, comsan, grewar3, lirplo, purher1]|
|XC770506.ogg_6 |[bcnher, brwowl1, comgre, comsan, grewar3, lirplo, purher1]|
|XC833987.ogg_3 |[bcnher, brwowl1, comgre, comsan, grewar3, lirplo, purher1]|
|XC858110.ogg_10|[bcnher, brwowl1, comgre, comsan, grewar3, lirplo, purher1]|
|XC453302.ogg_10|[bcnher, brwowl1, comgre, comsan, grewar3, lirplo]         |
|XC305518.ogg_19|[asbfly, putbab1]                                          |
|XC347938.ogg_0 |[asbfly, comior1]                              

In [61]:
# now let's see what the most frequent predictions are in the dataset
from pyspark.ml.fpm import FPGrowth

fp = FPGrowth(itemsCol="predicted", minSupport=0.01, minConfidence=0.01)
model = fp.fit(predictions)
model.freqItemsets.orderBy(F.desc("freq")).show(truncate=False)
model.associationRules.orderBy(F.desc("support")).show(truncate=False)

+------------------------------------------+----+
|items                                     |freq|
+------------------------------------------+----+
|[asbfly]                                  |337 |
|[grewar3]                                 |8   |
|[grewar3, brwowl1]                        |8   |
|[lirplo]                                  |8   |
|[lirplo, grewar3]                         |8   |
|[lirplo, grewar3, brwowl1]                |8   |
|[lirplo, comgre]                          |8   |
|[lirplo, comgre, grewar3]                 |8   |
|[lirplo, comgre, grewar3, brwowl1]        |8   |
|[lirplo, comgre, brwowl1]                 |8   |
|[lirplo, comgre, comsan]                  |8   |
|[lirplo, comgre, comsan, grewar3]         |8   |
|[lirplo, comgre, comsan, grewar3, brwowl1]|8   |
|[lirplo, comgre, comsan, brwowl1]         |8   |
|[lirplo, brwowl1]                         |8   |
|[lirplo, comsan]                          |8   |
|[lirplo, comsan, grewar3]                 |8   |


In [63]:
# prediction?
model.transform(predictions).orderBy(F.size("prediction").desc()).show()

+---------------+--------------------+----------+
|             id|           predicted|prediction|
+---------------+--------------------+----------+
|XC453302.ogg_10|[bcnher, brwowl1,...| [purher1]|
| XC134896.ogg_0|            [asbfly]|        []|
| XC134896.ogg_1|            [asbfly]|        []|
| XC134896.ogg_2|            [asbfly]|        []|
| XC134896.ogg_3|            [asbfly]|        []|
| XC134896.ogg_4|            [asbfly]|        []|
| XC134896.ogg_5|            [asbfly]|        []|
| XC164848.ogg_2|            [asbfly]|        []|
| XC175797.ogg_0|            [asbfly]|        []|
| XC175797.ogg_1|            [asbfly]|        []|
| XC175797.ogg_2|            [asbfly]|        []|
| XC175797.ogg_5|            [asbfly]|        []|
| XC267680.ogg_5|            [asbfly]|        []|
| XC267681.ogg_0|            [asbfly]|        []|
|XC267681.ogg_15|            [asbfly]|        []|
|XC267681.ogg_19|            [asbfly]|        []|
|XC267681.ogg_21|            [asbfly]|        []|
